In [13]:
from tensorflow.keras.applications import ResNet50V2, EfficientNetB0, DenseNet121
from tensorflow.keras.layers import Input
from data.datasets.image_classification import cifar100
from tensorflow.keras.applications import DenseNet121
from data.datasets.image_classification import cats_dogs
import tensorflow as tf

def CE(y_true, y_pred):
    """
    cross-entropy
    """
    return tf.math.negative(tf.reduce_sum(tf.math.log(y_pred) * y_true, axis=-1))

def identity(y_true, y_pred):
    """
    the identity transformation
    """
    return y_pred

image_size = (32, 32, 3)
inputs = Input(image_size)
dnet = DenseNet121(input_tensor=inputs, include_top=False, weights=None, pooling=None)
inp = tf.keras.layers.Input((32, 32, 3))
noise = tf.random.normal(tf.shape(inp))
print(tf.shape(inp))
x = dnet(inp)
y = dnet(noise)
x = tf.keras.layers.Flatten()(x)
y = tf.keras.layers.Flatten()(y)
dense = tf.keras.layers.Dense(100, activation=None)

x = dense(x)
outp_loss = dense(y)

outp = tf.keras.layers.Activation(tf.nn.softmax)(x)
sigma = (tf.reduce_mean(y * y, axis=-1) / tf.reduce_mean(noise * noise, axis=(1, 2, 3)))
# next do y, y
outputs = {'crossentropy': outp, 'loss_s': tf.reduce_sum(tf.expand_dims(sigma, -1) * y * y, axis=-1)}

model = tf.keras.models.Model(inputs=inp, outputs=outputs)

opt = tf.keras.optimizers.Nadam(learning_rate=1e-4,
                                beta_1=0.9, beta_2=0.999,
                                epsilon=None, decay=0.99)

opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)

model.compile(loss={'crossentropy': CE, 'loss_s': identity},
                  loss_weights=[1, 1e-2],
                  optimizer=opt,
                  metrics={'crossentropy': ['categorical_accuracy'], 'loss_s': []}
                  )

print(model.summary())

KerasTensor(type_spec=TensorSpec(shape=(4,), dtype=tf.int32, name=None), inferred_value=[None, 32, 32, 3], name='tf.compat.v1.shape_13/Shape:0', description="created by layer 'tf.compat.v1.shape_13'")
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 tf.compat.v1.shape_12 (TFOpLam  (4,)                0           ['input_14[0][0]']               
 bda)                                                                                             
                                                                                                  
 tf.random.normal_6 (TFOpLambda  (None, 32, 32, 3)   0           ['tf.compat.v1.shape_12[

In [14]:
dset_dict = cifar100(image_size=(32, 32))

train = dset_dict['train'].shuffle(256).batch(256)
val = dset_dict['val'].shuffle(256).batch(256)

history = model.fit(train, validation_data=val, epochs=100)

Epoch 1/100
196/196 [==============================] - 169s 540ms/step - loss: 6.0380 - activation_6_loss: 4.1957 - tf.math.reduce_sum_5_loss: 184.2328 - activation_6_categorical_accuracy: 0.0837 - val_loss: 1044.4241 - val_activation_6_loss: 5.1059 - val_tf.math.reduce_sum_5_loss: 103931.8047 - val_activation_6_categorical_accuracy: 0.0123
Epoch 2/100
196/196 [==============================] - 102s 517ms/step - loss: 4.0446 - activation_6_loss: 3.6784 - tf.math.reduce_sum_5_loss: 36.6223 - activation_6_categorical_accuracy: 0.1560 - val_loss: 160.6182 - val_activation_6_loss: 5.1851 - val_tf.math.reduce_sum_5_loss: 15543.3135 - val_activation_6_categorical_accuracy: 0.0089
Epoch 3/100
196/196 [==============================] - 102s 520ms/step - loss: 3.4807 - activation_6_loss: 3.3280 - tf.math.reduce_sum_5_loss: 15.2698 - activation_6_categorical_accuracy: 0.2142 - val_loss: 1574.2231 - val_activation_6_loss: 4.9957 - val_tf.math.reduce_sum_5_loss: 156922.7656 - val_activation_6_cate